In [2]:
# 1
from google.colab import drive
drive.mount('/gdrive/')

Mounted at /gdrive/


In [3]:
# 2
import os 
import scipy 
import matplotlib.pyplot as plt 
import librosa
import librosa.display
import numpy as np
from scipy.signal import filtfilt
import warnings


# from logmmse import logmmse_from_file

warnings.filterwarnings("ignore")

In [4]:
# 3
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
# import catboost as cbt
from sklearn.neighbors import KNeighborsClassifier

In [5]:
# 4
def preProcessingBlock(file,a=1,plot=False):
    # Pre-emphasis
    signal,fs = librosa.load(file)
    signal=np.asarray(signal)
    pre_sig=filtfilt([1,-a],1,signal)
    #Silent Remove Removal
    return pre_sig, fs

In [ ]:
print(preProcessingBlock("/gdrive/My Drive/temp.wav"))

(array([ 0.        , -0.03790172,  0.01789165, ...,  0.00233766,
       -0.01157563,  0.        ]), 22050)


In [6]:
# 5
def framing(file,window='hann',frameLength=20,shiftFrame=25):
    # signal,fs = librosa.load(file)
    # signal=np.asarray(signal)
    signal, fs = preProcessingBlock(file)
    sigLen=len(list(signal))
    outFrames=[]
    N=int(frameLength*fs/1000)
    Nshift=int(shiftFrame*fs/1000)
    window=scipy.signal.hann(N)

    numFrames=int(np.floor((sigLen-N)/Nshift) +1 )
    
    for i in range(0,numFrames):
        sigSlice=signal[N*i:(i+1)*N]
        # frame=np.dot(window,sigSlice)
        frame = map(lambda x,y: x*y, window, sigSlice)
        outFrames.append(list(frame))
    return(outFrames, fs)

In [ ]:
temp, fs = framing("/gdrive/My Drive/temp.wav")
temp = np.array(temp)
print(temp, temp.shape, fs)

[[ 0.00000000e+00 -1.93217275e-06  3.64817218e-06 ... -1.53387785e-06
  -8.53783943e-07  0.00000000e+00]
 [-0.00000000e+00  9.56056295e-07 -7.13717929e-07 ...  5.00654520e-06
  -5.19896778e-07 -0.00000000e+00]
 [ 0.00000000e+00 -1.51263983e-06  3.80351191e-06 ... -4.43752273e-06
   7.47876529e-07 -0.00000000e+00]
 ...
 [ 0.00000000e+00  6.63674321e-07 -1.46760142e-07 ...  1.60983216e-06
  -9.95269893e-07  0.00000000e+00]
 [ 0.00000000e+00 -7.72266061e-07  5.49970414e-06 ...  1.93370568e-06
  -2.07988195e-07 -0.00000000e+00]
 [-0.00000000e+00 -4.77853620e-08  1.63587144e-06 ...  1.17646856e-07
  -6.33977430e-07  0.00000000e+00]] (1434, 441) 22050


In [7]:
# 6
from scipy.linalg import toeplitz
from scipy.signal import filtfilt
def autocorr(x):
    result = np.correlate(x, x, mode='full')
    # print(result)
    return result[(int)(result.size/2):]

def lpcCoeffs(data,orderLPC=13,window='hann',G=1):
    # data,fs = librosa.load(frame)
    # signal=np.asarray(data)
    sigLen=len((list(data)))
    window = scipy.signal.hann(sigLen,True)
    
    # sigLPC=np.dot(window,data)
    sigLPC = list(map(lambda x,y: x*y, window, data))
    # print(sigLPC)
    sigEnergy=sum(np.power(sigLPC,2))

    # LPC Analysis 
    autoCorr=autocorr(sigLPC)
    sigCorr=autoCorr/np.max(abs(autoCorr))

    A=sigCorr[1:orderLPC-1]
    A=toeplitz(A)
    A=(-1)*np.linalg.inv(A)

    r=sigCorr[2:orderLPC]
    rT=np.transpose(r)
  
    L=np.transpose(np.dot(rT,A))
    # Coefficeint Extraction
    # print(L)
    # lpcs=np.concatenate((np.array([1]),L),axis=0)
    lpcs = np.array([1] + list(L))
    # Signal Filtering 
    tempo = np.array([0] + list(-1*lpcs[2:]))

    filteredSig=filtfilt(tempo,G,sigLPC)

    #Residual Computation 
    lpResidual=sigLPC-filteredSig
    return(lpResidual)


def fileLPCRes(fi,orderLPC=13,window='hann',G=1):
    print("Hello")
    outFrames, fs = framing(fi)
    
    print(np.array(outFrames).shape)
    fileres=[]
    # Now, frame-wise LPC caluclation 
    for i in range(0,len(outFrames)):
        frame_lpres=lpcCoeffs(np.asarray(outFrames[i]))
        fileres.append(frame_lpres)

    return np.asarray(fileres), fs

In [ ]:
temp, fs = fileLPCRes("/gdrive/My Drive/temp.wav")
temp = np.array(temp)
print(temp, temp.shape)

Hello
(1434, 441)
[[-1.05879118e-22  1.21779179e-06  2.46313089e-06 ... -1.67379390e-06
  -5.89441255e-07 -7.94093388e-23]
 [ 1.27054942e-21  5.90134984e-06 -3.65605620e-06 ...  4.67951460e-06
  -1.54794350e-06 -8.47032947e-22]
 [ 0.00000000e+00 -1.45771296e-07  7.58471258e-08 ... -9.71085678e-07
   9.90381797e-07  5.29395592e-23]
 ...
 [ 0.00000000e+00 -3.78625355e-07  1.77178436e-06 ...  5.84188043e-08
   1.90415827e-07  7.94093388e-23]
 [ 1.32348898e-22 -1.26572729e-06  2.29602533e-06 ... -1.63628588e-06
   1.00181258e-06  5.29395592e-23]
 [-0.00000000e+00 -3.57761749e-08  8.26779494e-08 ...  7.49233036e-08
   5.36596458e-08  1.65436123e-24]] (1434, 441)


In [8]:
# 7
def mfcc(y,sr=22050, S=None, n_mfcc=15, dct_type=2, norm='ortho',flag=False):
    # y,sr = librosa.load(file)
    mfccs = librosa.feature.mfcc(y=y,sr=sr, n_mfcc=n_mfcc)

    if(flag):
        print('Plotting the spectrogram')
        plt.figure(figsize=(10, 4))
        librosa.display.specshow(mfccs, x_axis='time')
        plt.colorbar()
        plt.title('MFCC')
        plt.tight_layout()
        plt.show()

    return(mfccs)

In [ ]:
print(mfcc(temp[1]).shape)

(15, 1)


In [9]:
# 8
X = []
Y = []
test_paths = []

In [13]:
# FRAME BY FRAME
# 12
import os, random
random.seed(42)
positive_path = "/gdrive/My Drive/speech/spasmodic dysphonia_temp"

for fi in os.listdir(positive_path):
  path = os.path.join(positive_path, fi)
  if random.random() >= 0.6:
    test_paths.append((path, 1))
    continue
  temp, fs = fileLPCRes(path)
  temp = np.array(temp)
  # temp_signal, fs = preProcessingBlock(path)
  # peaks = get_peaks(temp_signal, 22050, 10 * 0.0001, 100)
  # print(peaks)
  # jitt = get_jitt(peaks)
  # shim = get_shim_2(peaks)
  # print(temp, temp.shape, fs)

  hnrs = params(path)
  for arr, hnr in zip(temp, hnrs):
    # mf = list(mfcc(arr)) + [jitt, shim]
    mf = list(mfcc(arr)) + [hnr]
    X.append(mf)
    Y.append(1)

Hello
(1653, 441)
Hello
(1296, 441)
Hello
(974, 441)
Hello
(1216, 441)
Hello
(1057, 441)
Hello
(1018, 441)
Hello
(999, 441)
Hello
(967, 441)
Hello
(737, 441)
Hello
(937, 441)
Hello
(1020, 441)
Hello
(1920, 441)
Hello
(905, 441)
Hello
(724, 441)
Hello
(1175, 441)
Hello
(1116, 441)
Hello
(1008, 441)
Hello
(928, 441)
Hello
(1465, 441)


In [ ]:
# CONCAT ALL FRAMES

# import os, random
positive_path = "/gdrive/My Drive/speech/spasmodic dysphonia_temp"

for fi in os.listdir(positive_path):
  path = os.path.join(positive_path, fi)
  if random.random() >= 0.6:
    test_paths.append((path, 1))
    continue
  temp, fs = framing(path)
  temp = np.array(temp)
  curr = []
  for arr in temp:
    curr.append(mfcc(arr))
  curr = np.array(curr).reshape(-1,15)
  # print(curr.reshape(-1,15).shape)
  Y.append(1)
  X.append(curr)

In [14]:
# FRAME BY FRAME
# 13
import os
negative_path = "/gdrive/My Drive/speech/Healthy_temp"

for fi in os.listdir(negative_path):
  path = os.path.join(negative_path, fi)
  if random.random() >= 0.6:
    test_paths.append((path, 0))
    continue
  temp, fs = fileLPCRes(path)
  temp = np.array(temp)
  # temp_signal, fs = preProcessingBlock(path)
  # peaks = get_peaks(temp_signal, 22050, 10 * 0.0001, 100)
  # # print(peaks)
  # jitt = get_jitt(peaks)
  # shim = get_shim_2(peaks)
  # print(temp, temp.shape, fs)

  hnrs = params(path)
  for arr, hnr in zip(temp, hnrs):
    # mf = list(mfcc(arr)) + [jitt, shim]
    mf = list(mfcc(arr)) + [hnr]
    X.append(mf)
    Y.append(0)

Hello
(892, 441)
Hello
(1209, 441)
Hello
(1338, 441)
Hello
(1552, 441)
Hello
(1824, 441)
Hello
(991, 441)
Hello
(1365, 441)
Hello
(1461, 441)
Hello
(1528, 441)
Hello
(1259, 441)
Hello
(1216, 441)
Hello
(1570, 441)
Hello
(1434, 441)
Hello
(1951, 441)
Hello
(1702, 441)
Hello
(1159, 441)
Hello
(1245, 441)
Hello
(1274, 441)
Hello
(1734, 441)
Hello
(1270, 441)
Hello
(1059, 441)
Hello
(1623, 441)
Hello
(1052, 441)
Hello
(1273, 441)


In [ ]:
# CONCAT ALL FRAMES
# NR

import os
negative_path = "/gdrive/My Drive/speech/Healthy_temp"

for fi in os.listdir(negative_path):
  path = os.path.join(negative_path, fi)
  if random.random() >= 0.6:
    test_paths.append((path, 0))
    continue
  temp, fs = framing(path)
  temp = np.array(temp)
  curr = []
  # print(temp, temp.shape, fs)
  for arr in temp:
    curr.append(mfcc(arr))
  curr = np.array(curr).reshape(-1, 15)
  X.append(curr)
  Y.append(0)

In [15]:
# 14
X = np.array(X)
Y = np.array(Y)
test = X.reshape((-1, 16))
print(test.shape, Y.shape)

(54096, 16) (54096,)


In [ ]:
# print(X[0].shape)
#NR
T = []
for i in X:
  print(i.flatten().shape)
  T.append(i.flatten())
T = np.array(T)

import tensorflow as tf
test = tf.keras.preprocessing.sequence.pad_sequences(T,truncating="post", padding="post")

In [ ]:
from sklearn.svm import SVC
# svm_model = SVC(kernel = 'rbf', C=1)
svm_model = SVC(kernel = 'rbf', C=1, class_weight="balanced")
svm_model = svm_model.fit(X=test, y=Y)

In [16]:
#15
# logistic_model = LogisticRegression()
logistic_model = LogisticRegression(class_weight="balanced")
logistic_model = logistic_model.fit(X=test, y=Y)

In [ ]:
# randomf_model = RandomForestClassifier(n_estimators=200, class_weight="balanced")
randomf_model = RandomForestClassifier(n_estimators=200)
randomf_model = randomf_model.fit(X=test, y=Y)

In [ ]:
xgb_model = XGBClassifier()
# xgb_model = XGBClassifier(class_weight='balanced')
xgb_model = xgb_model.fit(X=test, y=Y)

In [ ]:
from sklearn.mixture import GaussianMixture
gaussian_model = GaussianMixture(max_iter=500, n_components=2)
gaussian_model = gaussian_model.fit(test, Y)

In [ ]:
knn_model = KNeighborsClassifier(n_neighbors = 100)
knn_model = knn_model.fit(test, Y)

In [17]:
# 16
print(test.shape)
corrects = 0
incorr = 0

(54096, 16)


In [18]:
# 17
test_X = []
test_Y = []

In [19]:
# 18
for fi, label in test_paths:
  temp, fs = fileLPCRes(fi)
  temp = np.array(temp)
  # print(temp, temp.shape, fs)
  # dic = {}
  temper = []
  # temp_signal, fs = preProcessingBlock(fi)
  # peaks = get_peaks(temp_signal, 22050, 10 * 0.0001, 100)
  # # print(peaks)
  # jitt = get_jitt(peaks)
  # shim = get_shim_2(peaks)
  
  # for arr in temp:
  #   # fr = list(mfcc(arr)) + [jitt, shim] 
  #   temper.append(mfcc(arr))
  hnrs = params(path)
  for arr, hnr in zip(temp, hnrs):
    # mf = list(mfcc(arr)) + [jitt, shim]
    mf = list(mfcc(arr)) + [hnr]
    temper.append(mf)
  test_X.append(temper)
  test_Y.append(label)
    # print(np.array([fr]).shape)
    # lab = logistic_model.predict(np.array([fr]).reshape(1,15))
    # dic[lab[0]] = dic.get(lab[0], 0) + 1
  # lab = max(dic, key=dic.get)
  # if lab == label:
    # corrects += 1
  # else:
    # incorr += 1

Hello
(1295, 441)
Hello
(984, 441)
Hello
(791, 441)
Hello
(1281, 441)
Hello
(696, 441)
Hello
(1179, 441)
Hello
(835, 441)
Hello
(1466, 441)
Hello
(905, 441)
Hello
(987, 441)
Hello
(1314, 441)
Hello
(1644, 441)
Hello
(1622, 441)
Hello
(1404, 441)
Hello
(1763, 441)
Hello
(1705, 441)
Hello
(1596, 441)
Hello
(1246, 441)
Hello
(1400, 441)
Hello
(1524, 441)
Hello
(1195, 441)
Hello
(1624, 441)
Hello
(1042, 441)
Hello
(1452, 441)
Hello
(1051, 441)
Hello
(1168, 441)
Hello
(1498, 441)
Hello
(1120, 441)
Hello
(1470, 441)


In [24]:
# 19
pred = []
for fi, label in zip(test_X, test_Y):
  # temp, fs = framing(fi)
  temp = np.array(fi)
  # print(temp, temp.shape, fs)
  dic = {}
  for arr in temp:
    lab = logistic_model.predict(np.array([arr]).reshape(1,16))
    dic[lab[0]] = dic.get(lab[0], 0) + 1
  lab = max(dic, key=dic.get)
  pred.append(lab)
  if lab == label:
    corrects += 1
  else:
    incorr += 1

In [25]:
# 20
from sklearn.metrics import classification_report, accuracy_score
print(classification_report(test_Y, pred))
print(accuracy_score(test_Y, pred))

              precision    recall  f1-score   support

           0       0.93      0.72      0.81        18
           1       0.67      0.91      0.77        11

    accuracy                           0.79        29
   macro avg       0.80      0.82      0.79        29
weighted avg       0.83      0.79      0.80        29

0.7931034482758621


In [ ]:
print(incorr, corrects)

3 26


In [ ]:
## CONCATENATE
for fi, label in test_paths:
  temp, fs = framing(fi)
  temp = np.array(temp)
  # print(temp, temp.shape, fs)
  # dic = {}
  # for arr in temp:
  #   fr = mfcc(arr)
  #   # print(np.array([fr]).shape)
  #   lab = model.predict(np.array([fr]).reshape(1,15))
  #   dic[lab[0]] = dic.get(lab[0], 0) + 1
  # lab = max(dic, key=dic.get)

  curr = []
  # print(temp, temp.shape, fs)
  for arr in temp:
    curr.append(mfcc(arr))
  curr = np.array(curr).reshape(-1, 15)
  x_test = tf.keras.preprocessing.sequence.pad_sequences(np.array([curr.flatten()]),truncating="post", padding="post", maxlen=29265)
  print(x_test.shape)
  lab = model.predict(x_test)
  if lab == label:
    corrects += 1
  else:
    incorr += 1

In [10]:
# 9
import numpy as np
from scipy import signal
from math import log10 as log


def get_number_of_windows(total_len, win_len):

	if total_len % win_len == 0:
		return total_len / win_len
	return (total_len / win_len) + 1

def get_peaks(input_sound, freq, window_size_time,samples_to_check):
	peaks = []
	print(input_sound)
	input_sound = signal.detrend(input_sound)
	print("Input after detrending: ", input_sound)
	input_sound = np.absolute(input_sound)
	size_of_window = int(freq*window_size_time)
	size_of_signal = len(input_sound)

	num_windows = int(get_number_of_windows(size_of_signal, size_of_window))
	print(num_windows)
	for i in range(num_windows):
		base_index = i*size_of_window
		curr_window = input_sound[i*size_of_window:(i+1)*size_of_window]
		moving_avgs = []
		sum_ = 0
		for i, elem in enumerate(curr_window):
			sum_ += elem
			moving_avgs.append(sum_/(i+1))
		moving_avgs = np.array(moving_avgs)
		max_ind = moving_avgs.argmax()

		left_bound = max(0, max_ind-samples_to_check+base_index)
		right_bound = max_ind+samples_to_check+base_index

		subsignal_to_check = input_sound[left_bound: right_bound+1]
		# print(subsignal_to_check, subsignal_to_check.argmax())
		max_in_subsignal = subsignal_to_check.argmax()
		peaks.append((left_bound+max_in_subsignal, subsignal_to_check[max_in_subsignal]))
  
	return np.array(peaks)

def get_jitt(peaks):

	N = len(peaks)
	jitt_den = np.sum(peaks, axis=0)[0]
	jitta_numerator = 0
	for i in range(1, N):
		jitta_numerator += abs(peaks[i][0] - peaks[i-1][0])

	return (100 * N * jitta_numerator)/((N-1)*jitt_den)

def get_shim(peaks):

	N = len(peaks)
	shim_den = np.sum(peaks, axis=0)[1]
	shim_numerator = 0
	for i in range(1, N):
		shim_numerator += abs(peaks[i][1] - peaks[i-1][1])

	return (100 * N * shim_numerator)/((N-1)*shim_den)

def get_shim_2(peaks):

	N = len(peaks)
	shim_numerator = 0
	for i in range(1, N):
		shim_numerator += abs(log(peaks[i][1]/peaks[i-1][1]))

	return (20 * shim_numerator)/(N-1)




#### DOUBTS
# 1. Detrend and then absolute or reverse
# 2. Indices in jitt and shim are slighly confusing
# 3. Can there be duplicates in peaks


'''
Testing : 

window_size_time = 10 * 0.0001
# input_sound_vector = np.random.randint(10, size=(20))
# input_sound_vector = np.random.rand(300)
input_sound_vector = np.array([9, 0, 9, 8, 3, 0, 9, 0, 4, 1, 7, 1, 6, 0, 9, 7, 0, 1, 9, 1])
samples_to_check = 3
frequency = 5000


print("Input: ", input_sound_vector)
peaks = get_peaks(input_sound_vector, frequency)
print("Peak indexes and values: ", peaks)
jitt = get_jitt(peaks)
shim = get_shim(peaks)
print("Jitt value is : ", jitt)
print("Shim value is : ", shim)

'''

'\nTesting : \n\nwindow_size_time = 10 * 0.0001\n# input_sound_vector = np.random.randint(10, size=(20))\n# input_sound_vector = np.random.rand(300)\ninput_sound_vector = np.array([9, 0, 9, 8, 3, 0, 9, 0, 4, 1, 7, 1, 6, 0, 9, 7, 0, 1, 9, 1])\nsamples_to_check = 3\nfrequency = 5000\n\n\nprint("Input: ", input_sound_vector)\npeaks = get_peaks(input_sound_vector, frequency)\nprint("Peak indexes and values: ", peaks)\njitt = get_jitt(peaks)\nshim = get_shim(peaks)\nprint("Jitt value is : ", jitt)\nprint("Shim value is : ", shim)\n\n'

In [11]:
# 10
temp_signal, fs = preProcessingBlock("/gdrive/My Drive/temp.wav")
print(len(temp_signal))
window_size_time = 10 * 0.0001
samples_to_check = 5
peaks = get_peaks(temp_signal, 22050, 10 * 0.0001, 100)
print(peaks)
jitt = get_jitt(peaks)
shim = get_shim_2(peaks)
print("Jitt value is : ", jitt)
print("Shim value is : ", shim)

638879
[ 0.         -0.03989409  0.02428077 ... -0.00076317  0.00315938
  0.        ]
Input after detrending:  [ 7.36636634e-08 -3.98940117e-02  2.42808434e-02 ... -7.63215374e-04
  3.15933277e-03 -4.96098164e-08]
29040
[[2.60000000e+01 4.19624903e-02]
 [2.60000000e+01 4.19624903e-02]
 [2.60000000e+01 4.19624903e-02]
 ...
 [6.38877000e+05 3.15933277e-03]
 [6.38877000e+05 3.15933277e-03]
 [6.38877000e+05 3.15933277e-03]]
Jitt value is :  0.006887085276814954
Shim value is :  0.25721444982928887


In [12]:
#  11
!pip3 install Signal_Analysis
from Signal_Analysis.features.signal import get_F_0
from Signal_Analysis.features.signal  import get_HNR

def params(fi):
    temp, fs = framing(fi)
    # te, fs = fileLPCRes(fi)
    # print(len(temp), len(te))
    temp=np.asarray(temp)
    f0_frame=[]
    hnr_frame=[]
    for i in range(0,temp.shape[0]):
        hnrFrame=get_HNR(temp[i][:],fs)
        hnr_frame.append(hnrFrame)
    return(hnr_frame)
H=params("/gdrive/My Drive/temp.wav")
print(len(H))

     |████████████████████████████████| 378kB 5.9MB/s 
  Created wheel for Signal-Analysis: filename=Signal_Analysis-0.1.26-cp36-none-any.whl size=14538 sha256=ce0ecec8485046ecd495abe7872e3b12ceb151be869fda040422774997bbdc63
  Stored in directory: /root/.cache/pip/wheels/50/06/bb/04aa9ef50b93b5961b9817600ca1ff379f7091e63e09831655
Successfully built Signal-Analysis
1159
